In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
import ssl
from newspaper import Article, ArticleException
import requests
from multiprocessing import Pool

from urllib3.exceptions import NewConnectionError, MaxRetryError
from requests.exceptions import RequestException
import requests
from urllib3.exceptions import NewConnectionError, MaxRetryError
from bs4 import BeautifulSoup
import pandas as pd

import csv
import re 

ssl._create_default_https_context = ssl._create_unverified_context


In [2]:
def scrape_url_allsides(url_news):

    try:
        response = requests.get(url_news)
        soup = BeautifulSoup(response.content, 'html.parser')
        read_more_div = soup.find('div', class_='read-more-story')
        if read_more_div:
            full_story_url = read_more_div.find('a')['href']
            return full_story_url
        else:
            return None
    except (requests.exceptions.RequestException, requests.exceptions.ConnectionError, NewConnectionError, MaxRetryError):
        return None


In [3]:
def scrape_page(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        articles = soup.find_all('div', class_='views-row')
        data = []

        for article in articles:
            published_date = article.find('p', class_='search-result-publish-date').text.strip() if article.find('p', class_='search-result-publish-date') else 'NaN'
            title = article.find('h3', class_='search-result-title').text.strip() if article.find('h3', class_='search-result-title') else 'NaN'
            content_allsides = article.find('a', class_='search-result-body').text.strip() if article.find('a', class_='search-result-body') else 'NaN'
            url = article.find('a', class_='search-result-body')['href'] if article.find('a', class_='search-result-body') else 'NaN'
            news_channel = article.find('a', class_='search-result-source').text.strip() if article.find('a', class_='search-result-source') else 'NaN'
            bias = article.find('img')['src'].split('-')[-1].strip().split('.')[0] if article.find('img') else 'NaN'
            tags_element = article.find('p', class_='search-result-tags') if article.find('p', class_='search-result-tags') else None

            if tags_element:
                tags_html = tags_element.find('span', class_='field-content').prettify()  # Get the HTML content of the tags
                soup = BeautifulSoup(tags_html, 'html.parser')  # Create a new BeautifulSoup object with the HTML content
                tags = soup.find_all('a')  # Find all <a> tags within the HTML
                tag_texts = [tag.text.strip()  for tag in tags]  # Extract tag text from <a> tags
            else:
                tag_texts = None

            news_channel_url = scrape_url_allsides(url) if url != 'NaN' else 'NaN'
            print(news_channel_url)

            data.append({
                'Published Date': published_date,
                'Title': title,
                'AllSides Content': content_allsides,
                'URL': url,
                'News Channel': news_channel,
                "News Channel URL": news_channel_url,
                'Bias': bias,
                'Tags':tag_texts,
            })

            return data

    except (requests.exceptions.RequestException, requests.exceptions.ConnectionError, NewConnectionError, MaxRetryError, AttributeError):
        return []



In [48]:
def return_pages(args):
    n, url = args
    print(args)
    data = scrape_page(url)
    for i in range(1, n):
        url_new = url + "&page=" + str(i)
        data += scrape_page(url_new)

    return data

In [49]:
url_skeleton = "https://www.allsides.com/search?search={}&item_bundle=1&sort_by=node_created"

In [57]:
import asyncio
import aiohttp
import pandas as pd

async def fetch_page(session, url):
    async with session.get(url) as response:
        return await response.text()

async def return_pages_async(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_page(session, url) for url in urls]
        pages = await asyncio.gather(*tasks)
        return pages

async def scrape_all_pages(url_skeleton, topic, total_pages):
    final_url = url_skeleton.format(topic)
    urls = [final_url] + [final_url + f"&page={i}" for i in range(1, total_pages + 1)]
    return await return_pages_async(urls)


In [ ]:
async def main():
    url_skeleton = "https://www.allsides.com/search?search={}&item_bundle=1&sort_by=node_created"
    topic = "abortion"
    total_pages = 500
    
    pages = await scrape_all_pages(url_skeleton, topic, total_pages)
    data = []
    for page in pages:
        # Process each page and extract data
        # Example: data.extend(process_page(page))
        pass
    
    df = pd.DataFrame(data)
    df['topic'] = topic
    return df

# Run the asynchronous event loop
df = await main()


In [54]:
final_url = url_skeleton.format("abortion")
data = []

for url in [(500, final_url)] + [(500, final_url + f"&page={i}") for i in range(1, 500)]:
    # Call return_pages function for the current URL
    scraped_data = return_pages(url)
    data.extend(scraped_data)  # Append the scraped data to the data list

# Create DataFrame from the collected data
df = pd.DataFrame(data)
df['topic'] = "abortion"


(500, 'https://www.allsides.com/search?search=abortion&item_bundle=1&sort_by=node_created')
https://mediaite.com/trump/trump-absolutely-nukes-lindsey-graham-over-national-abortion-ban-disagreement
https://bostonglobe.com/2024/04/08/nation/president-biden-governor-healey-trump-abortion-stance
https://www.nationalreview.com/news/vatican-declares-transgender-surgeries-surrogacy-gender-theory-threats-to-human-dignity/
https://mediaite.com/news/breaking-trump-tells-execute-the-baby-lie-in-big-abortion-announcement
https://christianpost.com/voices/a-victory-for-religious-freedom-in-virginia.html
None
https://foxnews.com/politics/republicans-pro-life-advocates-urge-kansas-governor-sign-bill-making-coercive-abortion-felony
https://washingtonexaminer.com/news/2952528/charlie-kirk-slammed-for-calling-women-in-their-30s-not-at-their-prime
https://msnbc.com/rachel-maddow-show/maddowblog/trump-eyes-abortion-declaration-polls-show-lopsided-results-rcna146174
https://dispatch.com/story/news/politics/

KeyboardInterrupt: 

In [ ]:
final_url = url_skeleton.format("public health")
data = []

for url in [(500, final_url)] + [(500, final_url + f"&page={i}") for i in range(1, 500)]:
    # Call return_pages function for the current URL
    scraped_data = return_pages(url)
    data.extend(scraped_data)  # Append the scraped data to the data list

# Create DataFrame from the collected data
df_public_health = pd.DataFrame(data)
df_public_health['topic'] = "public health"


In [42]:
topics = ['abortion', 'defense-and-security', 'economy-and-jobs', 'energy', 'environment', \
          'foreign-policy', 'sustainability', 'gun-control-and-gun-rights', 'violence-america', \
            'healthcare', 'public-health', 'housing-and-homelessness', 'race', 'civil-rights']

In [43]:
def soup_basics(item):
    page = requests.get(item)
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

In [44]:
url = "https://www.allsides.com/search?search=abortion&item_bundle=1&sort_by=node_created"

In [66]:
def scrape_url_allsides(url_news):

    response = requests.get(url_news)
    soup = BeautifulSoup(response.content, 'html.parser')
    read_more_div = soup.find('div', class_='read-more-story')
    if read_more_div!=None:
        full_story_url = read_more_div.find('a')['href']
        return full_story_url
    else:
        return None

In [67]:
def scrape_page(url):
    print(url)
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except:
        print("An exception occurred")
        return None

In [68]:
def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Extract data from the page using BeautifulSoup
    # Example: titles = soup.find_all('h2', class_='title')
    articles = soup.find_all('div', class_='views-row')
    data = []

    for article in articles:
        
        published_date = article.find('p', class_='search-result-publish-date').text.strip() if article.find('p', class_='search-result-publish-date') else 'NaN'
        title = article.find('h3', class_='search-result-title').text.strip() if article.find('h3', class_='search-result-title') else 'NaN'
        content_allsides = article.find('a', class_='search-result-body').text.strip() if article.find('a', class_='search-result-body') else 'NaN'
        url = article.find('a', class_='search-result-body')['href'] if article.find('a', class_='search-result-body') else 'NaN'
        news_channel = article.find('a', class_='search-result-source').text.strip() if article.find('a', class_='search-result-source') else 'NaN'
        bias = article.find('img')['src'].split('-')[-1].strip().split('.')[0] if article.find('img') else 'NaN'
        tags_element = article.find('p', class_='search-result-tags') if article.find('p', class_='search-result-tags') else None

        if tags_element:
            tags_html = tags_element.find('span', class_='field-content').prettify()  # Get the HTML content of the tags
            soup = BeautifulSoup(tags_html, 'html.parser')  # Create a new BeautifulSoup object with the HTML content
            tags = soup.find_all('a')  # Find all <a> tags within the HTML
            tag_texts = [tag.text.strip()  for tag in tags]  # Extract tag text from <a> tags

        else:
            tag_texts = None

        news_channel_url = scrape_url_allsides(url) if url != 'NaN' else 'NaN'
        # if news_channel_url and (news_channel_url != "Nan" or news_channel_url != "" or news_channel_url != None or news_channel_url != "None"):
        #     news_channel_content = scrape_page(news_channel_url)
        # else:
        #     news_channel_content = "Nan"

        
        data.append({
            'Published Date': published_date,
            'Title': title,
            'AllSides Content': content_allsides,
            # 'Actual Content' : news_channel_content,
            'URL': url,
            'News Channel': news_channel,
            "News Channel URL": news_channel_url,
            'Bias': bias,
            'Tags':tag_texts,
        })
    
    return data

In [69]:
def return_all_pages(n,url):
    data = scrape_page(url)
    for i in range(1,n):
        url_new = url + "&page=" + str(i)
        data += scrape_page(url_new)

    return data

abortion_data = return_all_pages(100,url)



KeyboardInterrupt: 

In [70]:
df = pd.DataFrame(abortion_data)

print(df.head(10))


NameError: name 'abortion_data' is not defined

In [155]:
df.to_csv('output.csv', index=False)


In [131]:
df["News Channel"].value_counts

<bound method IndexOpsMixin.value_counts of 0                            Axios
1                 Associated Press
2       Wall Street Journal (News)
3           Fox News (Online News)
4     Multiple Writers - Lean Left
5                Fox 4 Kansas City
6                 Florida Politics
7                   Texas Standard
8                Los Angeles Times
9                NBC News (Online)
10               CBS News (Online)
11                           Axios
12           New York Times (News)
13               Fox 4 Kansas City
14             Florida Times-Union
15                       Patch.com
16            Catholic News Agency
17                 Washington Post
18                            KFGO
19                          WCAX 3
20                      Dan Schnur
21                       Patch.com
22                        Politico
23         "USA Today" Contributor
24                       The State
25                     ABC 11 WTVD
26                  Breitbart News
27         

In [ ]:
# should we use article headlines or the actual content of the news
# what should be the number of pages to be parsed through 
# what should be the number of topics to be considered?